## Comparison across geographical locations

### Import libraries

In [1]:
import os
import pandas as pd 
import numpy as np
from lxml import etree 
pd.set_option('display.max_columns', None)
pd.options.display.max_colwidth = 200
import os
import glob
import time
import re
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import matplotlib.patches as mpatches
import shutil
from IPython.display import clear_output
pd.set_option('display.max_rows', 500)
from manuf import manuf
from user_agents import parse
import dataframe_image as dfi
import tldextract

### Readin datasets of different geolocations

In [2]:
ny = pd.read_parquet("../datasets/ny_serv_withcrt.parquet")

In [3]:
ny_set = set(ny['server_name_indication'].tolist())

In [4]:
ff = pd.read_parquet("../datasets/ff_serv.parquet")

In [5]:
ff_set = set(ff['server_name_indication'].tolist())

In [6]:
sin = pd.read_parquet("../datasets/sin_serv.parquet")

In [7]:
sin_set = set(sin['server_name_indication'].tolist())

### Num. SNIs

In [8]:
len(ny_set)

1151

In [9]:
len(ff_set)

1149

In [10]:
len(sin_set)

1150

### SNIs with cert shared by all locations

In [11]:
ny_sub = ny[['server_name_indication', 'issuer_O', 'issuer_CN', 'cert_serialNum_from_x509af']].drop_duplicates()

ff_sub = ff[['server_name_indication', 'issuer_O', 'issuer_CN', 'cert_serialNum_from_x509af']].drop_duplicates()

sin_sub = sin[['server_name_indication', 'issuer_O', 'issuer_CN', 'cert_serialNum_from_x509af']].drop_duplicates()

In [12]:
ny_sub['cert_serialNum_from_x509af'].value_counts().shape

(841,)

In [13]:
ff_sub['cert_serialNum_from_x509af'].value_counts().shape

(833,)

In [14]:
sin_sub['cert_serialNum_from_x509af'].value_counts().shape

(815,)

In [15]:
comm_temp = pd.merge(ny_sub, ff_sub, how='inner', on=['server_name_indication', 'issuer_CN', 'issuer_O', 'cert_serialNum_from_x509af'])
comm_all = pd.merge(comm_temp, sin_sub, how='inner', on=['server_name_indication', 'issuer_CN', 'issuer_O', 'cert_serialNum_from_x509af'])

In [16]:
comm_all.shape

(1182, 4)

In [17]:
comm_all['server_name_indication'].value_counts().shape

(1087,)

### Differences

### Certificates not observed in New York dataset

In [22]:
ff_notny = nyff_temp.loc[nyff_temp['_merge'] == 'right_only'].drop(columns=['_merge'])

In [23]:
ff_notny_sin = pd.merge(ff_notny, sin_sub, how="outer", on=['server_name_indication', 'issuer_CN', \
                                                        'issuer_O', 'cert_serialNum_from_x509af'], indicator=True)

In [24]:
ff_notny_sin.loc[ff_notny_sin['_merge'] == "left_only"].groupby(['cert_serialNum_from_x509af', 'issuer_CN'])\
            ['server_name_indication'].agg(lambda x: x.nunique()).reset_index().shape

(99, 3)

### Certificates not observed in Frankfurt dataset

In [25]:
sinff_temp = pd.merge(sin_sub, ff_sub, how="outer", on=['server_name_indication', 'issuer_CN', \
                                              'issuer_O', 'cert_serialNum_from_x509af'], indicator=True)

In [26]:
sin_notff = sinff_temp.loc[sinff_temp['_merge'] == 'left_only'].drop(columns=['_merge'])

In [27]:
sin_notff_ny = pd.merge(sin_notff, ny_sub, how="outer", on=['server_name_indication', 'issuer_CN', \
                                                        'issuer_O', 'cert_serialNum_from_x509af'], indicator=True)

In [28]:
sin_notff_ny.loc[sin_notff_ny['_merge'] == "left_only"].groupby(['cert_serialNum_from_x509af', 'issuer_CN'])\
            ['server_name_indication'].agg(lambda x: x.nunique()).reset_index().shape

(82, 3)

### Certificates not observed in Singapore dataset

In [18]:
nyff_temp = pd.merge(ny_sub, ff_sub, how="outer", on=['server_name_indication', 'issuer_CN', \
                                              'issuer_O', 'cert_serialNum_from_x509af'], indicator=True)

In [19]:
ny_notff = nyff_temp.loc[nyff_temp['_merge'] == 'left_only'].drop(columns=['_merge'])

In [20]:
ny_notff_sin = pd.merge(ny_notff, sin_sub, how="outer", on=['server_name_indication', 'issuer_CN', \
                                                        'issuer_O', 'cert_serialNum_from_x509af'], indicator=True)

In [21]:
ny_notff_sin.loc[ny_notff_sin['_merge'] == "left_only"].groupby(['cert_serialNum_from_x509af', 'issuer_CN'])\
            ['server_name_indication'].agg(lambda x: x.nunique()).reset_index().shape

(106, 3)